In [ ]:
"""
use pickle: https://www.datacamp.com/community/tutorials/pickle-python-tutorial
h5py: https://www.pythonforthelab.com/blog/how-to-use-hdf5-files-in-python/
problems with the generator foooor sure: https://github.com/keras-team/keras/issues/1627
"""

In [1]:
import os
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

#Run from server

gpu4 = "GPU-71b4cdfc-e381-0b98-9b24-4fc06284b496" 
gpu5 = "GPU-99d0769a-9f86-4800-a40e-2320dddcf5d1" 
gpu6 = "GPU-7423cfb5-cff4-ec4d-7e96-ea6e1591d56f"
gpu7 = "GPU-c0a8738f-6dd0-1b78-c38f-4969fd3886a8"

os.environ["CUDA_VISIBLE_DEVICES"]= gpu6 + "," + gpu5

print(get_available_devices())

['/device:CPU:0', '/device:XLA_CPU:0', '/device:XLA_GPU:0', '/device:XLA_GPU:1', '/device:GPU:0', '/device:GPU:1']


In [2]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import h5py

In [3]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, Lambda, GlobalAveragePooling2D, concatenate
from tensorflow.keras.layers import UpSampling2D, Conv2D, Dropout, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop, Adam, SGD

In [ ]:
ImgDir = os.path.join("..","data","MICCAI_BraTS2020_TrainingData")
features_path = list()
labels_path = list()
count = 0
lim = 10

for folder in os.listdir(ImgDir):
    count +=1
    if 'Training' in folder and count<lim:
        new_dir = os.path.join(ImgDir,folder)
        data = os.listdir(new_dir)
        for files in data:
            if 'flair' in files:
                features_path.append(os.path.join(new_dir, files))
            if 'seg' in files:
                labels_path.append(os.path.join(new_dir, files))

print(len(features_path), features_path)
print(len(labels_path), labels_path)


In [ ]:
img_conc = []

for file in (labels_path):
    img = nib.load(file)
    imgarr = img.get_fdata()
    img_conc.append(imgarr)
    print(np.shape(img_conc))
    
img_conc = np.concatenate(img_conc,axis=2)

#img = nib.load(features_path[0])
imgarr = img.get_fdata()[:,:,10]
plt.imshow(imgarr, cmap ='gray')

In [ ]:
img_conc_nii=nib.Nifti1Image(img_conc, np.eye(4))

In [ ]:
nib.save(img_conc_nii, os.path.join('..','data','test_segmentation.nii.gz'))  

In [ ]:
images = nib.load(os.path.join('..','data','test_segmentation.nii.gz'))
imgarr = images.get_fdata()[:,:,10]
plt.imshow(imgarr, cmap ='gray')

In [4]:
X_nib = nib.load(os.path.join('..','data','test.nii.gz'))
y_nib = nib.load(os.path.join('..','data','test_segmentation.nii.gz'))

In [5]:
def generator():
    
    
    data_length = len(X_nib.get_fdata()[1,1,:])
    chunks = np.floor(data_length / batch_size)
    print('A')
    X = X_nib.get_fdata()
    y = y_nib.get_fdata()
    print('B')
    X /= 255
    y /= 255
    print('C')
    while 1:
        for i in range(int(chunks)):
            yield X[i*32:(i+1)*32], y[i*32:(i+1)*32]

my_generator = generator()

In [6]:
IMG_HEIGHT = 240
IMG_WIDTH = 240
smooth = 0.005

In [10]:
def get_model():
    in1 = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 1 ))

    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(in1)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)

    up1 = concatenate([UpSampling2D((2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up1)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)
    
    up2 = concatenate([UpSampling2D((2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv6 = Dropout(0.2)(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)

    up2 = concatenate([UpSampling2D((2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv7 = Dropout(0.2)(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
    segmentation = Conv2D(3, (1, 1), activation='sigmoid', name='seg')(conv7)

    model = Model(inputs=[in1], outputs=[segmentation])   

    model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics=['accuracy']
             )

    return model

In [11]:
model2=get_model()
model2.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 240, 240, 1) 0                                            
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 240, 240, 32) 320         input_2[0][0]                    
__________________________________________________________________________________________________
dropout_7 (Dropout)             (None, 240, 240, 32) 0           conv2d_14[0][0]                  
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 240, 240, 32) 9248        dropout_7[0][0]                  
_______________________________________________________________________________________

In [12]:
class printbatch(tf.keras.callbacks.Callback):
    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: start of batch {}; got log keys: {}".format(batch, keys))

    def on_train_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: end of batch {}; got log keys: {}".format(batch, keys))



pb = printbatch()
batch_size = 32
# modify the fit_generator call to include the callback pb
model2.fit(my_generator, epochs = 2, verbose=2, callbacks=[pb], validation_data=None, class_weight=None)

Epoch 1/2
...Training: start of batch 0; got log keys: []


InvalidArgumentError:  ConcatOp : Dimensions of inputs should match: shape[0] = [32,64,120,696] vs. shape[1] = [32,64,120,697]
	 [[node functional_3/concatenate_4/concat (defined at <ipython-input-12-27b925b579ff>:15) ]] [Op:__inference_train_function_4647]

Function call stack:
train_function


In [ ]:
model.fit(my_generator)